In [148]:
from pyomo.environ import AbstractModel, Set, Param, Var, Constraint, ConstraintList, Objective, NonNegativeIntegers, \
    NonNegativeReals, Reals, maximize
import os
os.environ['path'] = 'K:\\COIN-OR\\1.8.0\\win32-msvc12\\bin' + os.pathsep + os.environ['path']

In [454]:
target_cals = 1400
#target_macros = {'fat': 30, 'carbs': 25, 'protein': 45, 'fiber': 20.6}  # fat loss
#target_macros = {'fat': 15, 'carbs': 55, 'protein': 30, 'fiber': 30}  # muscle gain
target_macros = {'fat': 62.5, 'carbs': 5, 'protein': 32.5, 'fiber': 0}  # keto
targets = {'fat': target_macros['fat'] / 9 * target_cals / 100,
           'carbs': target_macros['carbs'] / 4 * target_cals / 100,
           'protein': target_macros['protein'] / 4 * target_cals / 100,
           'fiber': target_macros['fiber']}
print('targets =', targets)
#amounts = {
#    'banana': (0,2),
#    'apple': (0,2),
#    'egg': (1,2),
#    'yogurt/maple syrup': (0,2*227),
#    'celery': (0,65),
#    'peanut butter': (0*32,2*32),
#    'carrot': (0,65),
#    'hummus': (0*28,2*28),
#    'chicken': (0,16),
#    'smoothie': (0,2),
#    'protein powder': (0,2*30),
#    'broccoli': (0,3*78),
#    'cheese': (0,2),
#    'lettuce': (0,4*36),
#    'tomatoes': (0,4*75),
#    'greek dressing': (0,4*30),
#    'benefiber': (0,60),
#    'cottage cheese': (0,2),
#    'fruit cup': (0,2),
#}
amounts = {
#    'yogurt/maple syrup': 170,
    #'yogurt': (0,280),
#    'blueberries': 36,
#    'granola': 30,
    'celery': (65,130),#65,
    'peanut butter': (0,60),#1.2*32),
    'carrot': (65,130),#65,
    'hummus': (0,60),#1.2*28),
    'chicken': (0,16),
#    'smoothie': 1,
#    'protein powder': 30,
    'broccoli': (60,120),
    'cheese': (0,4),
    'lettuce': (54,108),
#    'tomatoes': (0,75),
    'ranch dressing': (15,30),
#    'gummies': 2,
    'cottage cheese': (0,2),
#    'fruit cup': 1,
    'bbq sauce': (0,30),
}
ints = ['egg', 'cheese', 'yogurt/maple syrup', 'smoothie', 'apple', 'banana', 'cottage cheese', 'gummies']

targets = {'fat': 97.22222222222223, 'carbs': 17.5, 'protein': 113.75, 'fiber': 0}


In [455]:
from collections import namedtuple
Food = namedtuple('Food', ['fat', 'carbs', 'protein', 'sodium', 'fiber', 'units'])
food = {
    'soymilk': Food(fat=1.99/243, carbs=10.01/243, protein=6/243, sodium=95/243, fiber=1/243, units='grams'),  # 1 cup = 243 g
    'egg': Food(fat=6.7, carbs=0.98, protein=6.09, sodium=88, fiber=0, units='large'),  # 1 large = 61 g
    'yogurt/maple syrup': Food(fat=0.01/170+0.0198, carbs=20.41/170, protein=18.01/170, sodium=67/170, fiber=0, units='grams yogurt'),  # 1 container = 170 g
# 0%    'yogurt': Food(fat=0.01/170, carbs=0, protein=18.01/170, sodium=67/170, fiber=0, units='grams'),  # 1 container = 170 g
    'yogurt': Food(fat=11/227, carbs=7/227, protein=20/227, sodium=80/227, fiber=0, units='grams'),  # 1 container = 170 g
    'blueberries': Food(fat=0.0033, carbs=0.1449, protein=0.0074, sodium=0.0001, fiber=0.0024, units='grams'),  # 1 container = 170 g
    'yogurt/blueberries/maple syrup': Food(fat=0.19/170, carbs=28.52/170, protein=18.42/170, sodium=68/170, fiber=1.3/170, units='grams yogurt'),  # 1 container = 170 g
    'celery': Food(fat=0.0017, carbs=0.0297, protein=0.0069, sodium=0.8, fiber=0.016, units='grams'),  # 100 g
    'peanut butter': Food(fat=25/30, carbs=6/30, protein=9/30, sodium=2/30, fiber=2/30, units='grams'),  # 2 tbsp = 32 g
    'carrot': Food(fat=0.0024, carbs=0.0958, protein=0.0093, sodium=0.69, fiber=0.028, units='grams'),  # 100 g
    'hummus': Food(fat=2.5/14, carbs=2/14, protein=1/14, sodium=62.5/14, fiber=0.5/14, units='grams'),  # 2 tbsp = 28 g
    'chicken': Food(fat=1.071, carbs=0, protein=9.306, sodium=22.2, fiber=0, units='ounces'),  # 100 g
    'broccoli': Food(fat=0.64/156, carbs=11.2/156, protein=3.72/156, sodium=64/156, fiber=5.2/156, units='grams'),  # 1/2 cup = 78 g
    'cheese': Food(fat=6, carbs=0, protein=7, sodium=190, fiber=0, units='sticks'),  # 1 stick
    'lettuce': Food(fat=0.0015, carbs=0.0287, protein=0.0136, sodium=0.28, fiber=0.013, units='grams'),  # 1 cup = 36 g
    'tomatoes': Food(fat=0.002, carbs=0.0389, protein=0.0088, sodium=0.05, fiber=0.012, units='grams'),  # 1 cup = 149 g
    'bell peppers': Food(fat=0.003, carbs=0.0603, protein=0.0099, sodium=0.04, fiber=0.021, units='grams'),  # 1 cup = 149 g
    'vinaigrette': Food(fat=(1+0)/2, carbs=(0+0.1703)/2, protein=(0+0.0049)/2, sodium=(0.02+0.23)/2, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'ranch dressing': Food(fat=5/30, carbs=4/30, protein=1/30, sodium=310/30, fiber=0, units='grams'),  # 1 tbsp = 16 g
    'protein powder': Food(fat=4/30, carbs=12/30, protein=11/30, sodium=0, fiber=12/30, units='grams'),
    'benefiber': Food(fat=0, carbs=7/7.4, protein=0, sodium=0, fiber=5/7.4, units='grams'),
    'almonds': Food(fat=0.4993, carbs=0.2155, protein=0.2115, sodium=0.01, fiber=0.125, units='grams'),
    'skirt steak': Food(fat=3.8, carbs=0, protein=9, sodium=0.189, fiber=0, units='ounce'),
    'salsa': Food(fat=0, carbs=2/30, protein=0, sodium=200/30, fiber=0, units='grams'),
    'cottage cheese': Food(fat=4.5, carbs=4, protein=13, sodium=400, fiber=0, units='cups'),
    'chicken parm': Food(fat=39, carbs=5.72, protein=70.5, sodium=358, fiber=1, units='serving'),
    'smoothie': Food(fat=0.92, carbs=51.22, protein=3.28, sodium=27, fiber=6.3, units='serving'),
    'tomato caper sauce': Food(fat=1.385, carbs=4.2075, protein=1.05, sodium=206.75, fiber=1.4, units='serving'),
    'salmon': Food(fat=1.8, carbs=0, protein=5.62, sodium=12, fiber=0, units='ounces'),
    'tilapia': Food(fat=0.79, carbs=0, protein=7.84, sodium=17, fiber=0, units='ounces'),
    'yogurt': Food(fat=0, carbs=9/227, protein=23/227, sodium=85/227, fiber=0, units='grams'),
    'fruit cup': Food(fat=0.09, carbs=25.13, protein=0.378, sodium=6.0, fiber=1.28, units='package and 20 g maple syrup'), # per 100 g drained, 1/2 cup = 77 g
    'chicken sausage': Food(fat=2, carbs=4, protein=5, sodium=160, fiber=0, units='link'),
    'pork sausage': Food(fat=0.248, carbs=0.0093, protein=0.1539, sodium=7.39, fiber=0, units='grams'), # 1 link = 25 g
    'almond milk': Food(fat=0.36, carbs=0.19, protein=0.19, sodium=23, fiber=0, units='ounces'),
    'banana': Food(fat=0.39, carbs=26.95, protein=1.29, sodium=1, fiber=3.1, units='medium'),
    'apple': Food(fat=0.31, carbs=25.13, protein=0.47, sodium=2, fiber=4.4, units='medium'),
    'breaded chicken': Food(fat=1.68, carbs=5.81, protein=10.09, sodium=118, fiber=0.18, units='ounces'),
    'pork chop': Food(fat=1.24, carbs=0, protein=8.10, sodium=19, fiber=0, units='ounces'),
    'chicken meatballs': Food(fat=2.3, carbs=1.31, protein=4.18, sodium=117, fiber=0.48, units='meatball w/ sauce'),
    'cauliflower': Food(fat=0.0045, carbs=0.0411, protein=0.0184, sodium=0, fiber=0.023, units='grams'),
    'asparagus': Food(fat=0.0022, carbs=0.0411, protein=0.0240, sodium=0, fiber=0.020, units='grams'),
    'green beans': Food(fat=0.0028, carbs=0.0788, protein=0.0189, sodium=0, fiber=0.032, units='grams'),
    'canned green beans': Food(fat=0.0041, carbs=0.0419, protein=0.0105, sodium=0, fiber=0.019, units='grams'),
    'canned corn': Food(fat=0.0122, carbs=0.1434, protein=0.0229, sodium=0, fiber=0.020, units='grams'),
    'creamed corn': Food(fat=0.0042, carbs=0.1813, protein=0.0174, sodium=0, fiber=0.012, units='grams'),
    'gummies': Food(fat=0, carbs=3.5, protein=0, sodium=0, fiber=2, units='gummies'),
    'granola': Food(fat=6/30, carbs=18/30, protein=3/30, sodium=105/30, fiber=2/30, units='grams'),
    'bbq sauce': Food(fat=0, carbs=6/32, protein=0, sodium=240/32, fiber=1/32, units='grams'),
}
for key, val in food.items():
    food[key]._replace(carbs=val.carbs - val.fiber)

In [456]:
model = AbstractModel()

In [457]:
model.F = Set(initialize=list(amounts.keys()))
model.N = Set(initialize=['fat', 'carbs', 'protein', 'sodium', 'fiber'])
model.Food = Set(initialize=list(food.keys()))

In [458]:
def a_init(model, f, n):
    return getattr(food[f], n)
model.a = Param(model.Food, model.N, within=NonNegativeReals, initialize=a_init)

In [459]:
model.x = Var(model.F, within=NonNegativeReals)
model.x_fat = Var(within=Reals)
model.x_carbs = Var(within=Reals)
model.x_protein = Var(within=Reals)

In [460]:
def obj_rule(model):
    return 9 * model.x_fat + 4 * model.x_protein + 4 * model.x_carbs
    #return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F)
model.obj = Objective(rule=obj_rule)
#model.obj = Objective(rule=obj_rule, sense=maximize)

In [461]:
def sodium_rule(model):
    return sum(model.x[f] * model.a[f, 'sodium'] for f in model.F) <= 2000
model.sodium_constraint = Constraint(rule=sodium_rule)

In [462]:
def fiber_rule(model, f):
    return sum(model.x[f] * model.a[f, 'fiber'] for f in model.F) >= targets['fiber']
#model.fiber_constraint = Constraint(model.F, rule=fiber_rule)

In [463]:
def x_fat_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) - model.x_fat <= targets['fat']
model.x_fat_lb_constraint = Constraint(rule=x_fat_lb_rule)

In [464]:
def x_fat_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'fat'] for f in model.F) + model.x_fat >= targets['fat']
model.x_fat_ub_constraint = Constraint(rule=x_fat_ub_rule)

In [465]:
def x_carbs_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) - model.x_carbs <= targets['carbs']
model.x_carbs_lb_constraint = Constraint(rule=x_carbs_lb_rule)

In [466]:
def x_carbs_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'carbs'] for f in model.F) + model.x_carbs >= targets['carbs']
model.x_carbs_ub_constraint = Constraint(rule=x_carbs_ub_rule)

In [467]:
def x_protein_lb_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) - model.x_protein <= targets['protein']
model.x_protein_lb_constraint = Constraint(rule=x_protein_lb_rule)

In [468]:
def x_protein_ub_rule(model):
    return sum(model.x[f] * model.a[f, 'protein'] for f in model.F) + model.x_protein >= targets['protein']
model.x_protein_ub_constraint = Constraint(rule=x_protein_ub_rule)

In [469]:
#model.tomatoes_constraint = Constraint(rule=lambda model: model.x['tomatoes'] - 75 / 72 * model.x['lettuce'] == 0)
#model.dressing_constraint = Constraint(rule=lambda model: model.x['greek dressing'] - 30 / 72 * model.x['lettuce'] == 0)
#model.protein_powder_lb_constraint = Constraint(rule=lambda model: model.x['protein powder'] - 30 * model.x['smoothie'] >= 0)
#model.protein_powder_ub_constraint = Constraint(rule=lambda model: model.x['protein powder'] - 45 * model.x['smoothie'] <= 0)
#model.fruit_cup_constraint = Constraint(rule=lambda model: model.x['cottage cheese'] - model.x['fruit cup'] == 0)

In [470]:
from pyomo.opt import SolverFactory
opt = SolverFactory('cbc')
instance = model.create_instance()
for f in instance.F:
    if isinstance(amounts[f], tuple):
        instance.x[f].setlb(int(amounts[f][0]))
        instance.x[f].setub(int(amounts[f][1]))
        instance.x[f].value = instance.x[f].lb
    else:
        instance.x[f] = amounts[f]
        instance.x[f].fixed = True
    if food[f].units in ['grams', 'ounces'] or f in ints:
        instance.x[f].domain = NonNegativeIntegers
import prettytable
pt_var = prettytable.PrettyTable(['Food', 'Lower', 'Upper', 'Initial'])
pt_fix = prettytable.PrettyTable(['Food', 'Value'])
for f in instance.F:
    if not instance.x[f].fixed:
        pt_var.add_row([f.title(), instance.x[f].lb, instance.x[f].ub, instance.x[f].value])
    else:
        pt_fix.add_row([f.title(), instance.x[f].value])
print(pt_var)
print(pt_fix)
print(f'Integers: {[f for f in instance.F if instance.x[f].domain is NonNegativeIntegers]}')
results = opt.solve(instance)
print(results)

+----------------+-------+-------+---------+
|      Food      | Lower | Upper | Initial |
+----------------+-------+-------+---------+
|     Carrot     |   65  |  130  |    65   |
|   Bbq Sauce    |   0   |   30  |    0    |
|     Hummus     |   0   |   60  |    0    |
|     Cheese     |   0   |   4   |    0    |
|     Celery     |   65  |  130  |    65   |
|    Chicken     |   0   |   16  |    0    |
|    Lettuce     |   54  |  108  |    54   |
|    Broccoli    |   60  |  120  |    60   |
| Cottage Cheese |   0   |   2   |    0    |
| Ranch Dressing |   15  |   30  |    15   |
| Peanut Butter  |   0   |   60  |    0    |
+----------------+-------+-------+---------+

Integers: ['carrot', 'bbq sauce', 'hummus', 'cheese', 'celery', 'chicken', 'lettuce', 'broccoli', 'cottage cheese', 'ranch dressing', 'peanut butter']

Problem: 
- Name: unknown
  Lower bound: 93.20797912
  Upper bound: 93.20797912
  Number of objectives: 1
  Number of constraints: 8
  Number of variables: 15
  Number of n

In [471]:
fat = carbs = protein = fiber = sodium = 0
import prettytable
import textwrap
pt = prettytable.PrettyTable(['Food', 'Value'])
for f in sorted(instance.F):
    value = instance.x[f].value
    fat += value * instance.a[f, 'fat']
    carbs += value * instance.a[f, 'carbs']
    protein += value * instance.a[f, 'protein']
    fiber += value * instance.a[f, 'fiber']
    sodium += value * instance.a[f, 'sodium']
    units = food[f].units
    if f == 'yogurt/maple syrup':
        units = f'{units}, {20*value//170} g maple syrup'
    pt.add_row([f, '\n'.join(textwrap.wrap(f'{value} {units}', 20))])

cals = 9 * fat + 4 * carbs + 4 * protein

print(pt)
print(f'Total Calories = {int(cals)}, Fat = {fat:4.1f}, Carbs = {carbs:.1f}, Protein = {protein:5.1f}, Sodium = {int(sodium):4d}, Fiber = {fiber:4.1f}')
print(f'   Target Cals = {target_cals}, Fat = {targets["fat"]:.1f}, Carbs = {targets["carbs"]:.1f}, Protein = {targets["protein"]:.1f}, Sodium < 2000, Fiber = {targets["fiber"]:.1f}')
print(f'Total Macros:  Fat = {fat*9/cals*100:4.1f}%, Carbs = {carbs*4/cals*100:4.1f}%, Protein = {protein*4/cals*100:4.1f}%')
print(f'     Targets:  Fat = {target_macros["fat"]:4.1f}%, Carbs = {target_macros["carbs"]:4.1f}%, Protein = {target_macros["protein"]:4.1f}%')

+----------------+------------+
|      Food      |   Value    |
+----------------+------------+
|   bbq sauce    | 0.0 grams  |
|    broccoli    | 60.0 grams |
|     carrot     | 65.0 grams |
|     celery     | 65.0 grams |
|     cheese     | 4.0 sticks |
|    chicken     | 5.0 ounces |
| cottage cheese |  1.0 cups  |
|     hummus     | 57.0 grams |
|    lettuce     | 54.0 grams |
| peanut butter  | 60.0 grams |
| ranch dressing | 15.0 grams |
+----------------+------------+
Total Calories = 1488, Fat = 97.1, Carbs = 40.2, Protein = 113.3, Sodium = 1821, Fiber = 11.6
   Target Cals = 1400, Fat = 97.2, Carbs = 17.5, Protein = 113.8, Sodium < 2000, Fiber = 0.0
Total Macros:  Fat = 58.7%, Carbs = 10.8%, Protein = 30.5%
     Targets:  Fat = 62.5%, Carbs =  5.0%, Protein = 32.5%
